# 080420 Guardar tweets en tiempo real del servicio de transmisión de Twitter con Tweepy y consultar los datos con Pandas

https://github.com/jasonqng/twitter-streaming-python/blob/master/twitter_streaming.ipynb

In [119]:
%matplotlib inline
import tweepy
import pandas as pd
import json
import numbers
import re
import os.path
import datetime

pd.options.display.max_colwidth = 400
pd.options.display.max_rows = 25
pd.options.display.max_columns = None

In [120]:
consumer_key = 'koO4XqTuWFr5ADGcE8kjIkVoU'
consumer_key_secret = '3F4sk9qU8zbKBROuLPUUj1uvE2YuhseXPe0ahMQoivg4icN5bL'
access_token = '1230251564616515586-2KqPsCG2mIJp3irRjENgHpCfQUxTUg'
access_token_secret = '6PJfMtYGY7w6csiIX9m1S5jFEKNZ3hE9PVkHKeN1S14iM'

auth = tweepy.OAuthHandler(consumer_key, consumer_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

Hacemos una prueba con los tweets de mi propia cuenta:

In [121]:
tweets_raw_data = []
public_tweets = api.home_timeline()
for tweet in public_tweets:
    print (tweet.text)
    tweets_raw_data.append(tweet)

Desde #Colmed rechazamos proyecto que busca suspender #EUNACOM por 2 https://t.co/20OIh7fhlR Emergencia Sanitaria p… https://t.co/N4EetC4Jig
Dra. Siches: "Es muy necesario focalizar las vacunas de la influenza en los grupos de riesgo" #HoyEsNoticiaCNN
Dra. Siches: "Tal como dijo @SochinfOficial, existe un problema real en la capacidad de testeo, una limitante para… https://t.co/SK3pwhAr5Z
Dra. Siches: "Lamentablemente, existen algunos centros que han presentado alguna limitación de elementos de protecc… https://t.co/x2OHRmh1gj
Dra. Siches: "La mesa social COVID no maneja las determinaciones sobre las cuarentenas, por lo que no podría decir… https://t.co/RAoM35EBr2
Dra. Siches: "Frente una pandemia debemos mirar lo que pasó en otros países, para evaluar que podemos implementar e… https://t.co/uRqUajzPst
Dra. Siches: "Tenemos que ser muy precavidos con este ánimo triunfalista sobre la situación del país. Sabemos que e… https://t.co/bhcz3Cdgl3
Dra. Siches: "Desde nuestro juicio es necesar

In [122]:
tweets_raw_data[0]._json

{'created_at': 'Thu Apr 09 00:00:00 +0000 2020',
 'id': 1248037911733256192,
 'id_str': '1248037911733256192',
 'text': 'Desde #Colmed rechazamos proyecto que busca suspender #EUNACOM por 2 https://t.co/20OIh7fhlR Emergencia Sanitaria p… https://t.co/N4EetC4Jig',
 'truncated': True,
 'entities': {'hashtags': [{'text': 'Colmed', 'indices': [6, 13]},
   {'text': 'EUNACOM', 'indices': [54, 62]}],
  'symbols': [],
  'user_mentions': [],
  'urls': [{'url': 'https://t.co/20OIh7fhlR',
    'expanded_url': 'http://xn--aos-7ma.La',
    'display_url': 'años.La',
    'indices': [69, 92]},
   {'url': 'https://t.co/N4EetC4Jig',
    'expanded_url': 'https://twitter.com/i/web/status/1248037911733256192',
    'display_url': 'twitter.com/i/web/status/1…',
    'indices': [117, 140]}]},
 'source': '<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str'

In [123]:

# https://github.com/tweepy/tweepy/blob/v3.5.0/docs/streaming_how_to.rst
class MyStreamListener(tweepy.StreamListener):
    """ Options you can set by passing to the MyStreamListener object:
        limit: int, how many tweets to capture
        print_output: bool, whether to print the tweet to screen
        save_output: bool, whether to save the tweet data to a csv file
        filename: str, the filename to name the saved output, by default it's file.csv
        include_rts: bool, whether to capture retweets
        strict_text_search: bool, ocasionally, stream will capture a tweet that doesn't actually include the search query
            set to True to filter out these "accidental" tweets
        search_terms: str or array, pass in the search query or an array of terms you want to use for filtering
            if strict_text_search = True. Script checks and turns any string into array of strings
    """
    def __init__(self,limit=20,print_output=True,save_output=True,
                 filename='file.csv',include_rts=True,strict_text_search=False,
                 search_terms=None):
        self.df = pd.DataFrame()
        self.limit = limit
        self.counter = 0
        self.print_output = print_output
        self.header=False
        self.save_output=save_output
        self.filename=filename
        self.include_rts=include_rts
        self.strict_text_search = strict_text_search
        self.search_terms = search_terms

    def on_data(self, data):
        d = {}
        decoded = json.loads(data)
        # full list of fields you can collect: https://dev.twitter.com/overview/api/tweets
        tweet_fields_to_collect = ['created_at','id','text','source','favorite_count','coordinates','lang','place','retweet_count','retweeted','truncated']
        user_fields_to_collect = ['name','screen_name','location','id_str','statuses_count','followers_count','friends_count','favourites_count','description']
        if self.strict_text_search:
            
            
            if not isinstance(self.search_terms, list):
                
                
                self.search_terms = re.findall(r"[\w']+", self.search_terms)
                
                
            if not any(term.lower() in decoded['text'].lower() for term in self.search_terms):
                
                
                print ("skipped")
                print (decoded['text'])
                return True
        for k,v in decoded.items():
            if k in tweet_fields_to_collect:
                if isinstance(v, numbers.Number):
                    v = str(v)
                try:
                    d['tweet_' + k.strip()] = v
                except:
                    print ("Failure collecting tweet field", v.encode('ascii', 'ignore'))
            if k=='user':
                for user_k,user_v in v.items():
                    if user_k in user_fields_to_collect:
                        if isinstance(user_v, numbers.Number):
                            user_v = str(user_v)
                        try:
                            d[user_k.strip()]=user_v
                        except:
                            print ("Failure collecting user field",user_v.encode('ascii', 'ignore'))
            if k=='retweeted_status':
                for retweet_k,retweet_v in v.items():
                    if retweet_k in tweet_fields_to_collect:
                        if isinstance(retweet_v, numbers.Number):
                            retweet_v = str(retweet_v)
                        try:
                            d['retweet_'+retweet_k.strip()]=retweet_v
                        except:
                            print ("Failure collecting retweet field",user_v.encode('ascii', 'ignore'))
        if not self.include_rts:
            if ('retweet_text' in d and len(d['retweet_text'])>0) or d['tweet_text'].startswith('RT @'):
                return True
        tweet_df = pd.DataFrame(d, index=[0])
        frames = [self.df, tweet_df]
        self.df = pd.concat(frames)
        self.counter+=1
        if self.print_output:
            try:
                print(decoded['text'])
            except:
                print("Failure outputting tweet text",decoded['text'].encode('ascii', 'ignore'))
        if self.counter>=self.limit:
            print("finished collecting %s tweets, ending" % self.limit)
            if self.include_rts and 'retweet_text' in self.df.columns:
                self.df = self.df[['tweet_' + x for x in tweet_fields_to_collect] + user_fields_to_collect + ['retweet_' + x for x in tweet_fields_to_collect]]
            else:
                self.df = self.df[['tweet_' + x for x in tweet_fields_to_collect] + user_fields_to_collect]
            self.df.rename(columns={'id_str':'user_id'},inplace=True)
            self.df.to_csv(self.filename, index=False, encoding='utf-8')
            return False
        else:
            return True
        
    def on_error(self, status_code):
        if status_code == 420:
            return False
        
    def on_disconnect(self, notice):
        print("disconnecting due to " + str(notice))

Algunas opciones que puedes personalizar:

    Ocasionalmente, la transmisión capturará un tweet que en realidad no incluye la consulta de búsqueda. Establezca strictly_text_search en True para filtrar estos tweets "accidentales".
    Establezca un límite para la cantidad de tweets que desea obtener.
    el nombre de archivo para el csv que se guardará se genera en la línea de nombre de archivo a continuación y se guardará en la misma carpeta en la que se encuentra este archivo .ipynb. Puede personalizarlo y colocarlo en otro lugar y nombrar algo más si lo necesita.



In [124]:



#search_terms = 'android,iphone'
usuario = ['633098103']

filename = '%s_%s.csv' % ('CM', datetime.datetime.now().strftime("%Y-%m-%d_%H.%M.%S"))
myStreamListener = MyStreamListener(limit=5,print_output=False,
                                    filename=filename,
                                    #search_terms=search_query,
                                    include_rts=False)
myStream = tweepy.Stream(auth, listener=myStreamListener)


#myStream.filter(track=[search_query])
myStream.filter(follow=usuario)

android_or_iphone_no_retweets_df = pd.read_csv(filename)


#usuario = ['14499829']
#stream.filter(follow=usuario)






KeyboardInterrupt: 

In [98]:
android_or_iphone_no_retweets_df

,tweet_created_at,tweet_id,tweet_text,tweet_source,tweet_favorite_count,tweet_coordinates,tweet_lang,tweet_place,tweet_retweet_count,tweet_retweeted,tweet_truncated,name,screen_name,location,user_id,statuses_count,followers_count,friends_count,favourites_count,description
0,Wed Apr 08 23:47:21 +0000 2020,1248034730097938434,Android 17 will definitely be in the Anime version of the Moro saga as proven in the manga #DragonBallSuper https://t.co/7mceZB16or,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",0,NaN,en,NaN,0,False,False,James Reilly,JRStartoonist92,NaN,1080028727545720832,519,26,81,10877,"Cartoonist, Writer, Gamer, Fan of Pop Culture Instagram: jamesreillystartoonist DeviantArt: JamesStarToonist"
1,Wed Apr 08 23:47:22 +0000 2020,1248034731708633088,@promofiverrgigs https://t.co/AxFnxEVdTR,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",0,NaN,und,NaN,0,False,False,NY,hania_yk,NaN,878018511930621952,636,11,196,5,Mrs Yousaf Khan👫\nMom of 1 cutie pie *Hania Yousaf Khan* 👪
2,Wed Apr 08 23:47:24 +0000 2020,1248034743842635777,@Mr_Nyanis @1daavi Boss how am I seing twitter for samsung galaxy and not android ? Abi samsung don scam me ?,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",0,NaN,en,NaN,0,False,False,Protect_ya_neck🌹,_Tifannnny,"Abuja, Nigeria",1095960359079497730,1432,135,229,1808,Love making friends || Relationship therapist || Ambitious || Medical student || A vibe || 👅💦 || @fcbarcelona
3,Wed Apr 08 23:47:31 +0000 2020,1248034771739049987,@promofiverrgigs https://t.co/qw5uaUsH2o,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",0,NaN,und,NaN,0,False,False,NY,hania_yk,NaN,878018511930621952,637,11,196,5,Mrs Yousaf Khan👫\nMom of 1 cutie pie *Hania Yousaf Khan* 👪
4,Wed Apr 08 23:47:33 +0000 2020,1248034779112460288,Androidアプリ 子供向け乗り物図鑑は、図書館で車の絵本ばかり読まれるお子様には特に喜んでもらえます。https://t.co/iIQEnPLaKs,"<a href=""http://twittbot.net/"" rel=""nofollow"">twittbot.net</a>",0,NaN,ja,NaN,0,False,False,子供向け乗り物図鑑,kodomozukan4u,NaN,633635575,48647,282,1031,18,子供向け乗り物図鑑は、Androidアプリで0-4歳までを対象とした乗り物図鑑アプリで厳選された86枚以上のかわいいイラストの乗り物の写真や動画を楽しめます。
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,Wed Apr 08 23:48:46 +0000 2020,1248035086936702976,"⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️\nTIKTOKK🔰\n\n🎉Ganhe até R$ 100, convidando seus amigos p/ o TikTok🎉\n\n👉 1 - Clique p/ baixar o T… https://t.co/hv4fA0nRbB","<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",0,NaN,pt,NaN,0,False,True,pra vocês ficarem com fome,Ssthefane1,"Norton Disney, England",2193197140,6355,3409,2987,7208,"Por mais difícil que seja, nunca deixe de acreditar em dias melhores 🌈\n\nDIGITE ESSE CÓDIGO 22166234 NO TIKTOK PRA ENTRA JÁ COM DINHEIRO NA CONTA. 💫👏😱 LINK👇"
46,Wed Apr 08 23:48:52 +0000 2020,1248035112882585600,あーあーあー私も人力やりてぇ…！\nパソコンなら内部音声録音できるけどワイスマホAndroidだから内部音声録音出来ねぇ…,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",0,NaN,ja,NaN,0,False,False,🐬ぬもた🦈 動画up,numota1024,NaN,1164858971456360448,2820,263,66,2420,同担他担さんウェルカム 腐も夢もいけるので苦手な方ご注意\n基本的に好きなように絵を落としていく\n🐬🦈の2人多め 🐙寮 腐は🐬🦈のリバと🦈🌹 🐬&🦈🐙\n声優さんも好き 別界隈は声優きらみゅん 成人済 誤字脱字マン
47,Wed Apr 08 23:48:53 +0000 2020,1248035113763590147,"@aguila_regio @theraggakramer @Soy_el_Aisaac @AnneOrPin Ya wey, no se te valla a calentar tu android!! 😂😂😂😂","<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",0,NaN,es,NaN,0,False,False,RICK,Rick80814362,Tejeringo el chico,1166078432087859202,1674,131,185,2222,"Ser mejor que el día anterior, vivir para la familia, estar ahí cada día."
48,Wed Apr 08 23:48:54 +0000 2020,1248035118666530818,うっわお前Twitter for Androidじゃん気持ちわりーー近づかんといてー,"<a href=""http://twittbot.net/"" rel=""nofollow"">twittbot.net</a>",0,NaN,ja,NaN,0,False,False,チームATfA 公式アカウント,teamATfA_sv,NaN,1084191270883344384,11275,14,10,2,シャドバチーム ATfA(アンチT

In [100]:


print (len(android_or_iphone_no_retweets_df[android_or_iphone_no_retweets_df['tweet_text'].str.contains("(?i)iphone")]))
android_or_iphone_no_retweets_df[android_or_iphone_no_retweets_df['tweet_text'].str.contains("(?i)iphone")][['tweet_text']]



2


,tweet_text
25,@IPhone_lD Lu iPhone tapi twitter for android\nLu pake HP nokia layar sentuh?
29,「ゆっくり」を、ゆっくり育てる、ゆっくり育成ゲームで、ゆっくり遊んでみませんか？\nAndroid https://t.co/djny2B3JSv\niPhone https://t.co/7uupNJoIRc\n#ゆっくり育てていってね https://t.co/qDCifs9vWg
